In [ ]:
!pip install requests
!pip install pandas
!pip install yfinance
!pip install secure-smtplib
!pip install email

In [15]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\vamsh\OneDrive\Desktop\Trading_Bot_version_1\Experiment_notebooks\fear-index-spy-trading-4fdeddcff8a3.json'

In [148]:
import datetime
import requests
import json
import io
import pandas as pd
from random import choice
from google.cloud import storage


bucket_name = 'historicaltradingdata'
file_name = 'fearindex_pull_ratio_2021.csv'


def read_csv_from_cloud_storage(bucket_name, file_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    
    content = blob.download_as_text()
    df = pd.read_csv(io.StringIO(content))
    
    return df

def update_and_replace_csv_in_cloud_storage(bucket_name, file_name, updated_df):
    
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    
    updated_content = updated_df.to_csv(index=False)
    blob.upload_from_string(updated_content)
    
    return "File updated and replaced successfully"

def get_data_for_current_day():

    USER_AGENTS = [
        # Chrome on Windows 10
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36",
        # Chrome on macOS
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36",
        # Chrome on Linux
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36",
        # Firefox on Windows
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0",
        # Firefox on Macos
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 12.4; rv:100.0) Gecko/20100101 Firefox/100.0",
    ]
    user_agent = choice(USER_AGENTS)
    headers = {
                "User-Agent": user_agent,
            }

    current_date = datetime.date.today() - datetime.timedelta(days=1)

    data = pd.DataFrame()

    try: 
        request = requests.get('https://production.dataviz.cnn.io/index/fearandgreed/graphdata/' + str(current_date), headers=headers)
        r = request.json()['fear_and_greed_historical']['data'][0]
        data['Date'] = [current_date]
        data['index'] = [r['y']]
        data['pull_call_ratio'] = [request.json()['put_call_options']['data'][0]['y']]

    except:
        print(f"Data not available for {current_date}")

    return data


## updating the data

updated_df = pd.concat([read_csv_from_cloud_storage(bucket_name, file_name),get_data_for_current_day()],axis=0).reset_index(drop=True)
updated_df['Date'] = pd.to_datetime(updated_df['Date'])
updated_df.drop_duplicates(inplace=True)
# update_and_replace_csv_in_cloud_storage(bucket_name, file_name, updated_df)



In [95]:
import yfinance as yf
class FearandGreedIndexStrategy:
    def __init__(self,fear_data):
        self.data = None
        self.fear_data = fear_data

     # Download data from yfinance
    def load_data(self, symbol, start_date, end_date):
        print(symbol)
        self.data = yf.download(symbol,start_date, end_date).reset_index()

    def fear_category(self,x):
        if x <= 25:
            return "Extreme fear"
        elif (x <= 50) and (x > 25):
            return "Fear"
        elif (x <= 75) and (x > 50):
            return "Greed"
        elif (x <= 100) and (x > 75):
            return "Extreme Greed"
        
    def calculate_rsi(self, data, period=21):
        Close_prices = data['index']
        price_changes = Close_prices.diff()

        gains = price_changes.mask(price_changes < 0, 0)
        losses = -price_changes.mask(price_changes > 0, 0)

        average_gain = gains.rolling(window=period).mean()
        average_loss = losses.rolling(window=period).mean()

        rs = average_gain / average_loss
        rsi = 100 - (100 / (1 + rs))

        return rsi
    
    # STOCHASTIC OSCILLATOR CALCULATION

    def get_stoch_osc(self, high, low, close, k_lookback, d_lookback):
        lowest_low = low.rolling(k_lookback).min()
        highest_high = high.rolling(k_lookback).max()
        k_line = ((close - lowest_low) / (highest_high - lowest_low)) * 100
        d_line = k_line.rolling(d_lookback).mean()
        return k_line, d_line

    
    def generate_signals(self):
        Complete_df = pd.merge(self.fear_data, self.data, on='Date', how='inner').set_index('Date')
        Complete_df['signal'] = 0
        
        Complete_df['k'] = self.get_stoch_osc(Complete_df['High'], Complete_df['Low'], Complete_df['Close'], 21, 6)[0]
        Complete_df['d']= self.get_stoch_osc(Complete_df['High'], Complete_df['Low'], Complete_df['Close'], 21, 6)[1]
        Complete_df['Fear_Strength_index'] = self.calculate_rsi(Complete_df, period=21)
        
        Complete_df.dropna(inplace=True)

        condition = (Complete_df[['k', 'd']] <= [20, 20]).sum(axis=1) >= 1
        condition &= Complete_df['pull_call_ratio'] > 1
        condition &= Complete_df['Fear_Strength_index'] <= 60

        Complete_df['signal'] = condition.astype(int)
        
        return Complete_df

In [98]:
# Initiate Backtesting and Load Data
strategy = FearandGreedIndexStrategy(fear_data = updated_df)
strategy.load_data('SPY', '2021-01-01' ,str(datetime.date.today()))
results = strategy.generate_signals()

SPY
[*********************100%***********************]  1 of 1 completed


In [99]:
results


,index,pull_call_ratio,Open,High,Low,Close,Adj Close,Volume,signal,k,d,Fear_Strength_index
Date,,,,,,,,,,,,
2021-02-09,75.400000,0.481976,389.609985,390.890015,389.170013,390.250000,376.130341,35551100,0,97.170584,91.664462,59.421365
2021-02-10,75.266667,0.472632,392.119995,392.279999,387.500000,390.079987,375.966461,59154400,0,90.837104,93.545157,59.362648
2021-02-11,76.000000,0.460165,391.239990,391.690002,388.100006,390.709991,376.573700,42913300,0,93.461030,96.200644,59.582310
2021-02-12,77.666667,0.459426,389.850006,392.899994,389.769989,392.640015,378.433868,50593300,0,98.944461,96.071083,60.072816
2021-02-16,73.933333,0.469509,393.959991,394.170013,391.529999,392.299988,378.106140,50972400,0,92.779831,95.494791,58.482987
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-12,80.085714,0.703036,446.390015,447.480011,444.910004,446.019989,446.019989,91924500,0,91.565441,81.996412,52.253458
2023-07-13,81.114286,0.669071,447.899994,450.380005,447.450012,449.559998,449.559998,72425200,0,95.726903,82.339410,51.671183
2023-07-14,80.000000,0.664635,450.480011,451.359985,448.489990,449.279999,449.279999,69761800,0,89.687713,84.805389,48.894009


## Email alert

In [125]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart


subject = " SPY stock market alert table"
sender = "tradingbotwichita@gmail.com"
recipients = ["tradingbotwichita@gmail.com", "bvnd.sai321@gmail.com"]
password = "dufscwypfsxebszu"

def send_email_with_dataframe(subject, dataframe, sender, recipients, password):
    # Convert the DataFrame to an HTML table
    html_table = dataframe.to_html(index=False)

    # Create a multipart message and set the appropriate headers
    message = MIMEMultipart()
    message["Subject"] = subject
    message["From"] = sender
    message["To"] = ', '.join(recipients)

    # Set the HTML content of the email
    email_body = f"""
    <html>
    <body>
    <h2>5-Day Fear_index Trading:</h2>
    {html_table}
    </body>
    </html>
    """
    message.attach(MIMEText(email_body, "html"))

    # Connect to the SMTP server and send the email
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp_server:
        smtp_server.login(sender, password)
        smtp_server.sendmail(sender, recipients, message.as_string())
    print("Message sent!")

# Example usage:
# Create a sample DataFrame or use your own 'dataframe' DataFrame
dataframe = results[['index', 'pull_call_ratio','Close','signal']].reset_index().tail(5)

# Send the email with the DataFrame table
send_email_with_dataframe(subject, dataframe, sender, recipients, password)


Message sent!


In [129]:
!pip freeze

anyio==3.7.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arrow==1.2.3
asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1670263926556/work
async-lru==2.0.2
attrs==23.1.0
Babel==2.12.1
backcall @ file:///home/conda/feedstock_root/build_artifacts/backcall_1592338393461/work
backports.functools-lru-cache @ file:///home/conda/feedstock_root/build_artifacts/backports.functools_lru_cache_1618230623929/work
beautifulsoup4==4.12.2
bleach==6.0.0
cachetools==5.3.1
cattrs==23.1.2
certifi==2023.5.7
cffi==1.15.1
charset-normalizer==3.1.0
colorama @ file:///home/conda/feedstock_root/build_artifacts/colorama_1666700638685/work
contourpy==1.0.7
cryptography==41.0.1
cycler==0.11.0
debugpy @ file:///C:/ci_310/debugpy_1642079916595/work
decorator @ file:///home/conda/feedstock_root/build_artifacts/decorator_1641555617451/work
defusedxml==0.7.1
exceptiongroup==1.1.1
executing @ file:///home/conda/feedstock_root/build_artifacts/executing_1667317341051/work
fastjson

In [134]:
!pip install google-cloud